In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.resnet import ResNet18, ResNet34
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

MAX_TRAINING = 30000

#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (_, _) = cifar.load_data()

x_train_max, y_train = get_random_data(x_train, y_train, 0, x_train.shape[0], MAX_TRAINING)
x_train = x_train_max / 255.0
print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))

Number of Training Samples: X=30000, Y=30000


In [3]:
from utilities.data_preprocessors import flip_vertical_np, flip_horizontal_np, rotate_np, flip_rotate, \
perform_swirl_transformation, perform_random_affine_transform, mixed_transformations

class TransformDataset(object):
    def scale(self, X, x_min=0, x_max=1):
        nom = (X-X.min(axis=0))*(x_max-x_min)
        denom = X.max(axis=0) - X.min(axis=0)
        denom[denom==0] = 1
        return x_min + nom/denom 
    def return_function(self, name, im):
        return getattr(self, 'if_' + name)(im)
    def if_flip_vertical_np(self, im):
        return self.scale(flip_vertical_np(im))
    def if_flip_horizontal_np(self, im):
        return self.scale(flip_horizontal_np(im))
    def if_rotate_np(self, im):
        return self.scale(rotate_np(im))
    def if_flip_rotate(self, im):
        return self.scale(flip_rotate(im))
    def if_perform_swirl_transformation(self, im):
        return self.scale(perform_swirl_transformation(im))
    def if_perform_random_affine_transform(self, im):
        return self.scale(perform_random_affine_transform(im))
    def if_mixed_transformations(self, im):
        return self.scale(mixed_transformations(im))

def get_random_shuffle_set(data1, data2, ratios=None):
    if ratios is None:
        ratios = {"flip_vertical_np":.1, 'flip_horizontal_np':.1, 'rotate_np':.15, 'flip_rotate':.2, \
                  'perform_swirl_transformation':.1, 'perform_random_affine_transform':.15, 'mixed_transformations':.2}
    _, H1, W1, C1 = data1.shape
    N, _ = data2.shape
    shuffles = np.random.randint(0, N, N)
    low = 0
    x_train_ret, y_train_ret, img_idx  = np.zeros((N, H1, W1, C1)), np.zeros((N,)), 0

    for key in ratios.keys():
        high = int(ratios.get(key) * N)
        data = shuffles[low:low+high]
        low += high
        for idxs in data:
            x_train_ret[img_idx] = TransformDataset().return_function(key, data1[idxs])
            y_train_ret[img_idx] = data2[idxs]
            img_idx += 1
    return x_train_ret, y_train_ret

In [4]:
x_train_ret, y_train_ret = get_random_shuffle_set(x_train, y_train)
np.save('../saved_numpy_data/cifar10_augmented_x_train.npy', x_train_ret)
np.save('../saved_numpy_data/cifar10_augmented_y_train.npy', y_train_ret)

In [5]:
# x_train_ret2 = np.load('../saved_numpy_data/cifar10_augmented_x_train.npy') 
# y_train_ret2 = np.load('../saved_numpy_data/cifar10_augmented_y_train.npy') 

In [6]:
print (x_train_ret.shape, y_train_ret.shape)

(30000, 32, 32, 3) (30000,)
